In [2]:
pip install openpyxl

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [4]:
#!/usr/bin/env python3
import os
import sys
from pathlib import Path
from datetime import datetime
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment

def save_files_to_xlsx(file_list, output_path="lista_plikow.xlsx"):
    """Zapisuje listę plików do pliku XLSX."""
    
    # Twórz nowy workbook
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Lista plików"
    
    # Nagłówki
    headers = ["Lp.", "Nazwa", "Typ", "Rozmiar", "Rozmiar (bytes)", "Ścieżka relatywna"]
    ws.append(headers)
    
    # Formatowanie nagłówków
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    
    for col in range(1, len(headers) + 1):
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal="center")
    
    # Dodaj dane
    for idx, file_info in enumerate(file_list, 1):
        ws.append([
            idx,
            file_info['name'],
            file_info['type'],
            file_info['size_formatted'],
            file_info['size_bytes'],
            file_info['relative_path']
        ])
    
    # Dostosuj szerokość kolumn
    column_widths = [5, 40, 10, 15, 15, 50]
    for col, width in enumerate(column_widths, 1):
        ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width = width
    
    # Zapisz plik
    wb.save(output_path)
    print(f"Lista plików zapisana do: {output_path}")

def collect_files_data(recursive=False):
    """Zbiera dane o plikach do listy."""
    
    try:
        script_dir = Path(__file__).parent.absolute()
    except NameError:
        script_dir = Path.cwd()
    
    file_list = []
    
    try:
        if recursive:
            # Rekursywnie
            for item in sorted(script_dir.rglob('*')):
                if item.is_file():
                    file_size = item.stat().st_size
                    size_formatted = format_file_size(file_size)
                    rel_path = str(item.relative_to(script_dir))
                    
                    file_list.append({
                        'name': item.name,
                        'type': 'PLIK',
                        'size_formatted': size_formatted,
                        'size_bytes': file_size,
                        'relative_path': rel_path
                    })
        else:
            # Tylko bieżący folder
            for item in sorted(script_dir.iterdir()):
                if item.is_file():
                    file_size = item.stat().st_size
                    size_formatted = format_file_size(file_size)
                    
                    file_list.append({
                        'name': item.name,
                        'type': 'PLIK',
                        'size_formatted': size_formatted,
                        'size_bytes': file_size,
                        'relative_path': item.name
                    })
                elif item.is_dir():
                    file_list.append({
                        'name': item.name,
                        'type': 'FOLDER',
                        'size_formatted': '-',
                        'size_bytes': 0,
                        'relative_path': item.name
                    })
    
    except PermissionError:
        print("Błąd: Brak uprawnień do odczytu folderu")
        return []
    except Exception as e:
        print(f"Błąd: {e}")
        return []
    
    return file_list

def format_file_size(file_size):
    """Formatuje rozmiar pliku."""
    if file_size < 1024:
        return f"{file_size} B"
    elif file_size < 1024**2:
        return f"{file_size/1024:.1f} KB"
    elif file_size < 1024**3:
        return f"{file_size/(1024**2):.1f} MB"
    else:
        return f"{file_size/(1024**3):.1f} GB"
def list_files_in_current_directory():
    """Wylistuje wszystkie pliki w folderze, w którym znajduje się skrypt."""
    
    # Pobierz ścieżkę do folderu ze skryptem
    try:
        script_dir = Path(__file__).parent.absolute()
    except NameError:
        # Jeśli __file__ nie jest zdefiniowane, użyj bieżącego folderu
        script_dir = Path.cwd()
    
    print(f"Folder: {script_dir}")
    print("-" * 50)
    
    # Liczniki
    file_count = 0
    folder_count = 0
    
    try:
        # Iteruj przez wszystkie elementy w folderze
        for item in sorted(script_dir.iterdir()):
            if item.is_file():
                file_size = item.stat().st_size
                size_str = format_file_size(file_size)
                
                print(f"[PLIK] {item.name} ({size_str})")
                file_count += 1
                
            elif item.is_dir():
                print(f"[FOLDER] {item.name}/")
                folder_count += 1
    
    except PermissionError:
        print("Błąd: Brak uprawnień do odczytu folderu")
        return
    except Exception as e:
        print(f"Błąd: {e}")
        return
    
    print("-" * 50)
    print(f"Znaleziono: {file_count} plików, {folder_count} folderów")

def list_files_recursively():
    """Wylistuje wszystkie pliki rekursywnie (z podfolderami)."""
    
    try:
        script_dir = Path(__file__).parent.absolute()
    except NameError:
        # Jeśli __file__ nie jest zdefiniowane, użyj bieżącego folderu
        script_dir = Path.cwd()
    
    print(f"Folder (rekursywnie): {script_dir}")
    print("-" * 50)
    
    file_count = 0
    
    try:
        # Rekursywnie przeszukaj wszystkie pliki
        for item in sorted(script_dir.rglob('*')):
            if item.is_file():
                # Oblicz relatywną ścieżkę
                rel_path = item.relative_to(script_dir)
                file_size = item.stat().st_size
                size_str = format_file_size(file_size)
                
                print(f"{rel_path} ({size_str})")
                file_count += 1
                
    except PermissionError:
        print("Błąd: Brak uprawnień do odczytu niektórych folderów")
        return
    except Exception as e:
        print(f"Błąd: {e}")
        return
    
    print("-" * 50)
    print(f"Znaleziono łącznie: {file_count} plików")

if __name__ == "__main__":
    print("=== LISTA PLIKÓW ===\n")
    
    # Sprawdź argumenty
    recursive = False
    save_to_xlsx = False
    
    if len(sys.argv) > 1:
        for arg in sys.argv[1:]:
            if arg == "-r":
                recursive = True
            elif arg == "-x" or arg == "--xlsx":
                save_to_xlsx = True
    
    if save_to_xlsx:
        # Zbierz dane i zapisz do XLSX
        file_list = collect_files_data(recursive)
        if file_list:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"lista_plikow_{timestamp}.xlsx"
            save_files_to_xlsx(file_list, output_file)
            print(f"Zapisano {len(file_list)} elementów do pliku {output_file}")
    else:
        # Wyświetl na konsoli
        if recursive:
            list_files_recursively()
        else:
            list_files_in_current_directory()
    
    print("\nUżycie: python skrypt.py [-r] [-x]")
    print("  -r     : wylistuj pliki rekursywnie (z podfolderami)")
    print("  -x     : zapisz do pliku XLSX")
    print("  --xlsx : zapisz do pliku XLSX")
    print("\nPrzykłady:")
    print("  python skrypt.py -x          # zapisz tylko bieżący folder do XLSX")
    print("  python skrypt.py -r -x       # zapisz rekursywnie do XLSX")

=== LISTA PLIKÓW ===

Folder: /Volumes/Public/yt download
--------------------------------------------------
[PLIK] "I want Llama3 to perform 10x with my private knowledge" - Local Agentic RAG w_ llama3.mp4 (78.9 MB)
[PLIK] "I want Llama3.1 to perform 10x with my private knowledge" - Self learning Local Llama3.1 405B.mp4 (84.0 MB)
[PLIK] "Make Agent 10x cheaper, faster & better?" -  LLM System Evaluation 101.mp4 (75.3 MB)
[PLIK] #08 | How To Deploy & Run Snowpark Python Inside Snowflake.mp4 (63.7 MB)
[PLIK] *Args and **Kwargs in Python.mp4 (10.2 MB)
[PLIK] .DS_Store (66.0 KB)
[PLIK] 10 Genius Apple Shortcuts to Simplify Your Life.mp4 (382.8 MB)
[PLIK] 10 Important Python Concepts In 20 Minutes.mp4 (55.2 MB)
[PLIK] 10 Python Functions That Will Simplify Your Life.mp4 (233.1 MB)
[PLIK] 10 automatyzacji, które OSZCZĘDZĄ Ci miesiące pracy.mp4 (43.5 MB)
[PLIK] 100+ Docker Concepts you Need to Know.mp4 (120.6 MB)
[PLIK] 1120bb8e420eb7b1f1578eb98dba.jpeg (796.4 KB)
[PLIK] 15 Minute Stretc

In [ ]:
all_files = list_files_in_current_directory()
all_files

In [1]:
# Lista plików w folderze - wersja dla Jupyter Notebook

import os
from pathlib import Path
from datetime import datetime
import pandas as pd

def format_file_size(file_size):
    """Formatuje rozmiar pliku."""
    if file_size < 1024:
        return f"{file_size} B"
    elif file_size < 1024**2:
        return f"{file_size/1024:.1f} KB"
    elif file_size < 1024**3:
        return f"{file_size/(1024**2):.1f} MB"
    else:
        return f"{file_size/(1024**3):.1f} GB"

def collect_files_data(folder_path=None, recursive=False):
    """Zbiera dane o plikach do listy."""
    
    if folder_path is None:
        # Użyj bieżącego folderu
        folder_path = Path.cwd()
    else:
        folder_path = Path(folder_path)
    
    print(f"Skanowanie folderu: {folder_path}")
    
    file_list = []
    
    try:
        if recursive:
            # Rekursywnie
            for item in sorted(folder_path.rglob('*')):
                if item.is_file():
                    file_size = item.stat().st_size
                    size_formatted = format_file_size(file_size)
                    rel_path = str(item.relative_to(folder_path))
                    
                    file_list.append({
                        'Lp': len(file_list) + 1,
                        'Nazwa': item.name,
                        'Typ': 'PLIK',
                        'Rozmiar': size_formatted,
                        'Rozmiar_bytes': file_size,
                        'Ścieżka': rel_path,
                        'Rozszerzenie': item.suffix.lower() if item.suffix else 'brak'
                    })
        else:
            # Tylko bieżący folder
            for item in sorted(folder_path.iterdir()):
                if item.is_file():
                    file_size = item.stat().st_size
                    size_formatted = format_file_size(file_size)
                    
                    file_list.append({
                        'Lp': len(file_list) + 1,
                        'Nazwa': item.name,
                        'Typ': 'PLIK',
                        'Rozmiar': size_formatted,
                        'Rozmiar_bytes': file_size,
                        'Ścieżka': item.name,
                        'Rozszerzenie': item.suffix.lower() if item.suffix else 'brak'
                    })
                elif item.is_dir():
                    file_list.append({
                        'Lp': len(file_list) + 1,
                        'Nazwa': item.name,
                        'Typ': 'FOLDER',
                        'Rozmiar': '-',
                        'Rozmiar_bytes': 0,
                        'Ścieżka': item.name,
                        'Rozszerzenie': 'folder'
                    })
    
    except PermissionError:
        print("❌ Błąd: Brak uprawnień do odczytu folderu")
        return []
    except Exception as e:
        print(f"❌ Błąd: {e}")
        return []
    
    return file_list

def lista_plikow(folder_path=None, recursive=False, save_xlsx=False, show_stats=True):
    """
    Główna funkcja do listowania plików.
    
    Args:
        folder_path: Ścieżka do folderu (None = bieżący folder)
        recursive: Czy skanować rekursywnie
        save_xlsx: Czy zapisać do pliku XLSX
        show_stats: Czy pokazać statystyki
    
    Returns:
        pandas.DataFrame z listą plików
    """
    
    print("📁 LISTA PLIKÓW")
    print("=" * 50)
    
    # Zbierz dane
    file_list = collect_files_data(folder_path, recursive)
    
    if not file_list:
        print("❌ Nie znaleziono plików lub błąd dostępu")
        return None
    
    # Utwórz DataFrame
    df = pd.DataFrame(file_list)
    
    # Wyświetl statystyki
    if show_stats:
        pliki_count = len(df[df['Typ'] == 'PLIK'])
        foldery_count = len(df[df['Typ'] == 'FOLDER'])
        total_size = df[df['Typ'] == 'PLIK']['Rozmiar_bytes'].sum()
        
        print(f"📊 STATYSTYKI:")
        print(f"   • Pliki: {pliki_count}")
        print(f"   • Foldery: {foldery_count}")
        print(f"   • Łączny rozmiar: {format_file_size(total_size)}")
        
        # Statystyki rozszerzeń
        if pliki_count > 0:
            ext_stats = df[df['Typ'] == 'PLIK']['Rozszerzenie'].value_counts()
            print(f"   • Najczęstsze rozszerzenia: {dict(ext_stats.head(5))}")
        
        print("=" * 50)
    
    # Zapisz do XLSX jeśli requested
    if save_xlsx:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"lista_plikow_{timestamp}.xlsx"
        
        # Utwórz sformatowaną wersję do zapisu
        df_save = df.copy()
        df_save = df_save.drop('Rozmiar_bytes', axis=1)  # Usuń kolumnę bajtów z eksportu
        
        try:
            df_save.to_excel(output_file, index=False, engine='openpyxl')
            print(f"💾 Zapisano do: {output_file}")
        except Exception as e:
            print(f"❌ Błąd zapisu: {e}")
    
    return df

# Funkcje pomocnicze dla szybkiego użycia
def lista_tutaj():
    """Wylistuj pliki w bieżącym folderze"""
    return lista_plikow()

def lista_rekursywnie():
    """Wylistuj pliki rekursywnie"""
    return lista_plikow(recursive=True)

def zapisz_liste():
    """Wylistuj pliki i zapisz do XLSX"""
    return lista_plikow(save_xlsx=True)

def zapisz_liste_rekursywnie():
    """Wylistuj pliki rekursywnie i zapisz do XLSX"""
    return lista_plikow(recursive=True, save_xlsx=True)

# Przykłady użycia:
print("🚀 GOTOWE DO UŻYCIA!")
print("\n📝 Przykłady:")
print("   df = lista_tutaj()                    # Lista plików w bieżącym folderze")
print("   df = lista_rekursywnie()              # Lista plików rekursywnie")
print("   df = zapisz_liste()                   # Lista + zapis do XLSX")
print("   df = zapisz_liste_rekursywnie()       # Lista rekursywnie + zapis do XLSX")
print("   df = lista_plikow('/ścieżka/folder')  # Lista plików w konkretnym folderze")
print("\n🔍 Po wykonaniu możesz:")
print("   df.head()                             # Pokaż pierwsze 5 wierszy")
print("   df[df['Typ'] == 'PLIK']               # Tylko pliki")
print("   df[df['Rozszerzenie'] == '.jpg']      # Tylko pliki JPG")
print("   df.groupby('Rozszerzenie').size()     # Grupuj według rozszerzenia")

🚀 GOTOWE DO UŻYCIA!

📝 Przykłady:
   df = lista_tutaj()                    # Lista plików w bieżącym folderze
   df = lista_rekursywnie()              # Lista plików rekursywnie
   df = zapisz_liste()                   # Lista + zapis do XLSX
   df = zapisz_liste_rekursywnie()       # Lista rekursywnie + zapis do XLSX
   df = lista_plikow('/ścieżka/folder')  # Lista plików w konkretnym folderze

🔍 Po wykonaniu możesz:
   df.head()                             # Pokaż pierwsze 5 wierszy
   df[df['Typ'] == 'PLIK']               # Tylko pliki
   df[df['Rozszerzenie'] == '.jpg']      # Tylko pliki JPG
   df.groupby('Rozszerzenie').size()     # Grupuj według rozszerzenia


In [2]:
all_files = zapisz_liste()
all_files

📁 LISTA PLIKÓW
Skanowanie folderu: /Users/mateusz/Desktop
📊 STATYSTYKI:
   • Pliki: 321
   • Foldery: 4
   • Łączny rozmiar: 71.2 GB
   • Najczęstsze rozszerzenia: {'.mp4': np.int64(315), 'brak': np.int64(3), '.jpeg': np.int64(2), '.ipynb': np.int64(1)}
💾 Zapisano do: lista_plikow_20250709_121930.xlsx


,Lp,Nazwa,Typ,Rozmiar,Rozmiar_bytes,Ścieżka,Rozszerzenie
0,1,"""I want Llama3 to perform 10x with my private ...",PLIK,78.9 MB,82699166,"""I want Llama3 to perform 10x with my private ...",.mp4
1,2,"""I want Llama3.1 to perform 10x with my privat...",PLIK,84.0 MB,88048726,"""I want Llama3.1 to perform 10x with my privat...",.mp4
2,3,"""Make Agent 10x cheaper, faster & better?"" - ...",PLIK,75.3 MB,78962324,"""Make Agent 10x cheaper, faster & better?"" - ...",.mp4
3,4,#08 | How To Deploy & Run Snowpark Python Insi...,PLIK,63.7 MB,66769881,#08 | How To Deploy & Run Snowpark Python Insi...,.mp4
4,5,*Args and **Kwargs in Python.mp4,PLIK,10.2 MB,10675321,*Args and **Kwargs in Python.mp4,.mp4
...,...,...,...,...,...,...,...
320,321,listing_all_files.ipynb,PLIK,76.4 KB,78235,listing_all_files.ipynb,.ipynb
321,322,uv: The Ultra-Fast Python Package Manager 🚀.mp4,PLIK,100.2 MB,105031164,uv: The Ultra-Fast Python Package Manager 🚀.mp4,.mp4
322,323,"“Fundamentals of Data Engineering Part 1"" - Fu...",PLIK,290.1 MB,304181425,"“Fundamentals of Data Engineering Part 1"" - Fu...",.mp4
323,324,𝙊𝙡𝙙 𝙍𝙖𝙙𝙞𝙤 𝙎𝙩𝙤𝙧𝙮 _ 𝙂𝙪𝙞𝙩𝙖𝙧 𝙇𝙤𝙛𝙞 _ 𝘾𝙖𝙛𝙚 𝙈𝙪𝙨𝙞𝙘 _ 𝙀...,PLIK,264.4 MB,277267483,𝙊𝙡𝙙 𝙍𝙖𝙙𝙞𝙤 𝙎𝙩𝙤𝙧𝙮 _ 𝙂𝙪𝙞𝙩𝙖𝙧 𝙇𝙤𝙛𝙞 _ 𝘾𝙖𝙛𝙚 𝙈𝙪𝙨𝙞𝙘 _ 𝙀...,.mp4
